Dans ce notebook, nous allons effectuer une "traduction" d'image à image en utilisant des CycleGAN. Nous proposons une méthode qui capture les caractéristiques d'images du base de données (MNIST) et applique ses caractéristiques dans un autre domaine de données (USPS), et ce sans exemple de paires pour l'entrainement (problème non-supervisé).

Les CycleGANs utilisent une perte de cycle constante pour permettre l'entrainement sans images en paires. En d'autres termes, ils permettent de traduire d'un domaine à un autre sans avoir mappé au préalable les données.

## Set up

In [1]:
# importation des données
import h5py

# tensorflow
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

# outils
import numpy as np
import matplotlib.pyplot as plt

# from IPython.display import clear_output

# mesure du temps
import time

In [2]:
# tfds.disable_progress_bar()
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Données

### MNIST

In [3]:
(mnist_images, mnist_labels), (mnist_images_test, mnist_labels_test) = tf.keras.datasets.mnist.load_data()

mnist_images = mnist_images.reshape(mnist_images.shape[0], mnist_images.shape[1], mnist_images.shape[2], 1).astype('float32')
mnist_images = (mnist_images - 127.5) / 127.5 # on normalise pour obtenir des valeurs dans l'intervalle [-1, 1]

mnist_images2 = np.ndarray(shape=(mnist_images.shape[0], mnist_images.shape[1], mnist_images.shape[2], 3))
for i in range(mnist_images.shape[0]):
    for j in range(mnist_images.shape[1]):
        for k in range(mnist_images.shape[2]):
            mnist_images2[i][j][k]=np.array([float(mnist_images[i][j][k]), float(mnist_images[i][j][k]), float(mnist_images[i][j][k])])

BUFFER_SIZE = mnist_images.shape[0]
BATCH_SIZE = 1
mnist_dataset = tf.data.Dataset.from_tensor_slices(mnist_images2).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

KeyboardInterrupt: 

### USPS

In [ ]:
def random_crop(image):
    cropped_image = tf.image.random_crop(image, size=[256, 256, 3])
    return cropped_image

def random_jitter(image):
    # on définit la taille à 256 x 256 x 3
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # on crop l'image au hasard à 256 x 256 x 3
    image = random_crop(image)
    # random mirroring
    image = tf.image.random_flip_left_right(image)
    return image

# def preprocess_image_train(image, label):
#     image = random_jitter(image)
#     image = normalize(image)
#     return image

# def preprocess_image_test(image, label):
#     image = normalize(image)
#     return image

In [ ]:
filename = "Data/usps.h5"
f = h5py.File(filename, 'r+')

train = f['train']
usps_images = np.array(train['data'])
usps_labels = np.array(train['target'])

test = f['test']
usps_images_test= np.array(test['data'])
usps_labels_test = np.array(test['target'])

usps_images = usps_images.reshape(usps_images.shape[0], int(np.sqrt(usps_images.shape[1])), int(np.sqrt(usps_images.shape[1])), 1).astype('float32')

usps_images2 = np.ndarray(shape=(usps_images.shape[0], usps_images.shape[1], usps_images.shape[2], 3))
for i in range(usps_images2.shape[0]):
    for j in range(usps_images.shape[1]):
        for k in range(usps_images.shape[2]):
            usps_images2[i][j][k]=np.array([float(usps_images[i][j][k]), float(usps_images[i][j][k]), float(usps_images[i][j][k])])

BUFFER_SIZE = usps_images.shape[0]
BATCH_SIZE = 1
usps_dataset = tf.data.Dataset.from_tensor_slices(usps_images2).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
sample_mnist = next(iter(mnist_dataset))
sample_usps = next(iter(usps_dataset))
sample_mnist = tf.image.resize(sample_mnist, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
sample_usps = tf.image.resize(sample_usps, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(221)
plt.title('mnist')
plt.imshow(sample_mnist[0], cmap='gray')

plt.subplot(222)
plt.title('mnist with random jitter')
plt.imshow(random_jitter(sample_mnist[0]), cmap='gray')

plt.subplot(223)
plt.title('usps')
plt.imshow(sample_usps[0], cmap='gray')

plt.subplot(224)
plt.title('usps with random jitter')
plt.imshow(random_jitter(sample_usps[0]), cmap='gray')

## Pix2Pix models

On importe le générateur et le discriminateur du modèle Pix2Pix en installant le package tensorflow_examples.

L'architecture du modèle utilisé ets très similaire à ce qui est utilisé dans le modèle Pix2Pix.

Il y a deux générateurs ($G$ et $F$) qui sont entrainés, et deux discriminateurs ($D_X$ et $D_Y$) :
* Le générateur $G$ apprend à transformer les images $X$ en image $Y$
* Le générateur $F$ apprend à transformer les images $Y$ en image $X$
* Le discriminateur $D_X$ apprend à différentier les vraies images $X$ des images générées $X$
* Le discriminateur $D_Y$ apprend à différentier les vraies images $Y$ des images générées $Y$


In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
to_usps = generator_g(sample_mnist)
to_mnist = generator_f(sample_usps)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_mnist, to_usps, sample_usps, to_mnist]
title = ['mnist', 'to usps', 'usps', 'to mnist']

for i in range(len(imgs)):
    plt.subplot(2, 2, i+1)
    plt.title(title[i])
    if i % 2 == 0:
        plt.imshow(imgs[i][0] * 0.5 + 0.5)
    else:
        plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Vrai mnist?')
plt.imshow(discriminator_x(sample_mnist)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Vrai usps?')
plt.imshow(discriminator_y(sample_usps)[0, ..., -1], cmap='RdBu_r')

plt.show()

## Fonctions de perte
Dans les CycleGANs, il n'y a pas de données couplés pour l'entrainement, donc il n'y a aucune garantie que l'input x et la target y sont couplés de manière sensée durant l'entrainement. Donc, pour apprendre à correctement mapper, on utilise une perte de consistance de cycle ("cycle consistency loss")  

In [ ]:
LAMBDA = 10

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss / 2

In [ ]:
def generator_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)

La consistance du cycle signifie que le résulat devrait être proche de l'input original. Par exemple, si on traduit une phrase de l'anglais au français, puis qu'on retraduit du français vers l'anglais, alors la phrase obtenue devrait être la même que la phrase originale.

Dans la perte de consistance de cycle, 
* le générateur $G$ crée une image (générée) $\hat{Y}$ à partir d'une vraie image $X$
* le générateur $F$ crée une image (cyclée) $\hat{X}$ à partir de l'image générée $\hat{Y}$
* on calcule l'erreur moyenne absolue entre $X$ et $\hat{X}$ (forward cycle consistency loss) et entre $Y$ et $\hat{Y}$ (backward cycle consistency loss)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return LAMBDA * loss1

In [ ]:
def identity_loss(real_image, same_image):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.5 * loss

### Fonctions d'opimisation

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored !!')

## Entrainement
La boucle d'entrainement est composée des 4 étapes suivantes : 
1. Obtenir les prédictions
2. Calculer la perte
3. Calculer les gradients par backpropagation
* Appliquer les gradients à la fonction d'optimisation

In [ ]:
EPOCHS = 40

In [ ]:
def generate_images(model, test_input):
    prediction = model(test_input)
    
    plt.figure(figsize=(12, 12))

    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
@tf.function
def train_step(real_x, real_y):
    
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.
    
        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)

        fake_x = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)
    
        # same_x and same_y are used for identity loss.
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)

        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # calculate the loss
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)
    
        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
        # Total generator loss = adversarial loss + cycle loss
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
    # Calculate the gradients for generator and discriminator
    generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)
  
    discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
    # Apply the gradients to the optimizer
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))
  
    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()

    n = 0
    for image_x, image_y in tf.data.Dataset.zip((mnist_dataset, usps_dataset)):
        image_x = tf.image.resize(image_x, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        image_y = tf.image.resize(image_y, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        image_x = tf.cast(image_x, tf.float32)
        image_y = tf.cast(image_y, tf.float32)

        train_step(image_x, image_y)
        if n % 10 == 0:
            print ('.', end='')
        n+=1

    clear_output(wait=True)
    # Using a consistent image (sample_horse) so that the progress of the model
    # is clearly visible.
    generate_images(generator_g, sample_mnist)

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in mnist_dataset.take(5):
    inp = tf.image.resize(inp, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    inp = tf.cast(inp, tf.float32)
    generate_images(generator_g, inp)